In [2]:
import numpy as np
import pandas as pd
import os

In [3]:
path = os.getcwd() + "/atribuna/aTribuna_completo"
content_file_array = []
classifier_array = []

In [4]:
def read_text(path, file_Name):
    with open(path, encoding="latin-1") as file:
        content_file = file.read()
        Build_DataFrame(content_file, file_Name)

In [5]:
def Build_DataFrame(content_file, file_Name):
    classifier =  GetClassifier(file_Name) 
    if(len(classifier) >= 3):        
        classifier_array.append(classifier)
        content_file_array.append(content_file)  

In [6]:
def GetClassifier(file_Name):
    name_split =  file_Name.split(".")
    first_name = name_split[0];
    return first_name[(len(first_name) - 3) : len(first_name)]

In [7]:
for file_Name in os.listdir(path):    
    if file_Name.endswith(".txt"):
        file_path = f"{path}\{file_Name}"
      
        # call read text file function
        read_text(file_path, file_Name)

FileNotFoundError: [WinError 3] O sistema não pode encontrar o caminho especificado: 'C:\\el\\fabio_helmer\\PI4\\NPL\\NLP_KNN/atribuna/aTribuna_completo'

In [ ]:
textDataDrame =  pd.DataFrame({"Text":content_file_array, "Class":classifier_array})


In [ ]:
textDataDrame.head(10)

# StopWords removing

In [8]:
import nltk

In [14]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [33]:
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [84]:
dict = {'Text' : ('carlos é legal so não mais que o rafa',
                  'joão foi mama todo mundo',
                  'gabriel é a porra de um cara foda',
                  'breno vai morre aos 20 anos se não dormi'),
        'Class' : ('1','2','3','4')
       }

In [81]:
df = pd.DataFrame.from_dict(dict)

In [85]:
returnDataFrameStopWords(df)

,0,1
0,"[breno, vai, morre, 20, anos, dormi]",4
1,"[gabriel, porra, cara, foda]",3
2,"[joão, mama, todo, mundo]",2
3,"[carlos, legal, so, rafa]",1


In [73]:
def returnDataFrameStopWords(dataFrame):
    stop_words = set(stopwords.words('portuguese'))
    dataReturn = pd.DataFrame()
    for row in dataFrame.itertuples(index=True, name='Pandas'):
        Text = row.Text
        Class = row.Class

        Text = addStopWordsToList(Text)

        linha = pd.Series([Text, Class])
        linha = pd.DataFrame([linha])
        dataReturn = pd.concat([linha, dataReturn], ignore_index=True)
        
    return dataReturn
        

In [80]:
def addStopWordsToList(texto):
    example_sent = texto
    word_tokens = word_tokenize(example_sent)

    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]

    filtered_sentence = []

    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    return filtered_sentence

# Imports do EDA

In [1]:
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve,classification_report,r2_score
from sklearn import model_selection as ms

In [4]:
# print da curva ROC
def plot_roc_curve(y_true, y_score, figsize=(10,6)):
    fpr, tpr, _ = roc_curve(y_true, y_score)
    plt.figure(figsize=figsize)
    auc_value = roc_auc_score(y_true, y_score)
    plt.plot(fpr, tpr, color='orange', label='ROC curve (area = %0.2f)' % auc_value)
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('Taxa de falso positivo')
    plt.ylabel('Taxa de verdadeiro positivo')
    plt.title('Curva ROC')
    plt.legend()
    plt.show()

In [3]:
#precision    recall  f1-score   support
print(classification_report(y_test, pred, labels=[1, 2, 3]))

NameError: name 'y_test' is not defined

In [5]:
# AUC
print('AUC: %0.2f' % roc_auc_score(y_test, pred))|

SyntaxError: invalid syntax (<ipython-input-5-61188af94a59>, line 2)

In [6]:
def estimativas(nome_col,nome_fill,column_df,coluna_filtro):
    
    filtros = []
    for item in coluna_filtro:
        if(item not in filtros ):
            filtros.append(item)
    
    for filtro in filtros:
        column_filter = column_df[coluna_filtro == filtro]
        mean = np.mean(column_filter)
        median = np.median(column_filter)
        sd = np.std(column_filter)
        var = np.var(column_filter)
        coef_var = (sd/mean)*100
        mad = median_absolute_deviation(column_filter)
        iqr = stats.iqr(column_filter)
        
        print(f'************** {nome_col} com {nome_fill} {filtro} ***************')
        print(f'media:  %.2f' % mean)
        print(f'mediana:  %.2f' % median)
        print(f'Desvio Padrao:  %.2f' % sd)
        print(f'Variancia:  %.2f' % var)
        print(f'Coeficiente de variacao:  %.2f' % coef_var)
        print(f'Desvio Absoluto Médio:  %.2f' % mad)
        print(f'Amplitude Interquartil:  %.2f' % iqr)

In [7]:
estimativas('salary','Graduação',df_original.salary,df_original.graduacao)

NameError: name 'df_original' is not defined